In [1]:
import cv2  # OpenCVで動画と画像処理
import torch  # PyTorchでYOLOv5モデル利用
import sys

### 1. YOLOv5モデルのロード

In [2]:
# ========= 1. YOLOv5モデルのロード =========
#   yolov5s.ptは自動でダウンロードされます
#   deviceを'cpu'に明示することでCPU専用実行になります
model = torch.hub.load(
    './yolov5',          # クローンしたYOLOv5リポジトリのパス
    'custom',            # カスタムモデル（yolov5s等も可）
    path='yolov5s.pt',   # 学習済みモデル（デフォルトでOK）
    source='local',      # ローカルファイルとして読み込む
    device='cpu'         # CPU強制
)

# ラベル（人、車…などの名前リスト）
class_names = model.names

YOLOv5 🚀 v7.0-429-g50313d30 Python-3.11.13 torch-2.2.2 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


### 2. 動画ファイルの読み込み

In [3]:

# ========= 2. 動画ファイルの読み込み =========
video_path = 'data/nihonbashi-short-mini.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("動画が開けません。ファイルパスを確認してください。")
    sys.exit(1)

### 3. 推論（物体検出）

YOLOv5の推定(modelメソッド)の返り値について（GPTの解説）
- 型:  yolov5.utils.general.Detections
- 座標形式ごとの Tensor を返す
    - results.xyxy
        [x1, y1, x2, y2, conf, cls] （左上と右下の座標）
        
        | index  | 内容                                  |
        | ------ | ------------------------------------- |
        | 0      | `x1` バウンディングボックスの左上の x座標 |
        | 1      | `y1` バウンディングボックスの左上の y座標 |
        | 2      | `x2` バウンディングボックスの右下の x座標 |
        | 3      | `y2` バウンディングボックスの右下の y座標 |
        | 4      | `conf` 検出確信度 (confidence score)       |
        | 5      | `cls` クラスID（0=person, 1=bicycle, ...） | 
        <br>
    - results.xywh
        [x_center, y_center, width, height, conf, cls]
   
        | index  | 内容                                  |
        | ------ | ------------------------------------- |
        | 0      | `x_center` バウンディングボックスの中心の x座標 |
        | 1      | `y_center` バウンディングボックスの中心の y座標 |
        | 2      | `width` バウンディングボックスの幅|
        | 3      | `height` バウンディングボックスの高さ |
        | 4      | `conf` 検出確信度 (confidence score)       |
        | 5      | `cls` クラスID（0=person, 1=bicycle, ...） |

    - results.xyxyn / results.xywhn
        上記を画像サイズで 0〜1に正規化した値

In [4]:
def recognition(frame):
    results = model(frame[..., ::-1])  # YOLOv5に画像を渡して検出
    return results

### 4. 結果をフレームに描画

In [5]:
def draw_info_on_mov(results, frame):
    for *box, conf, cls in results.xyxy[0].tolist():
        # 座標とラベル
        x1, y1, x2, y2 = map(int, box)
        label = class_names[int(cls)]
        score = float(conf)
        # 検出枠
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # ラベル
        cv2.putText(
            frame,
            f"{label}: {score:.2f}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )

### 描画ループ

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break  # 動画の最後でループ終了

    # ========= 3. 推論（物体検出） =========
    # OpenCVはBGR⇒YOLOはRGB
    results = recognition(frame)
    
    # ========= 4. 結果をフレームに描画 =========
    # 各物体ごとに検出ボックス＆ラベルを追加
    draw_info_on_mov(results, frame)
    
    # ========= 5. ウィンドウ表示 =========
    cv2.imshow("YOLOv5 Detection", frame)
    # qキーで途中終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ========= 6. 後処理 =========
cap.release()
cv2.destroyAllWindows()

: 

In [5]:
import cv2

# 入力動画を開く
cap = cv2.VideoCapture("./data/shinjuku.mp4")

# 動画の情報を取得
fps = cap.get(cv2.CAP_PROP_FPS)                       # フレームレート
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))       # 横幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))      # 縦幅
size = (width, height)

# 出力設定（ファイル名, FourCC, FPS, サイズ）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4形式
out = cv2.VideoWriter("output.mp4", fourcc, fps, size)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ここで処理を加えることも可能
    # 例: グレースケールに変換（保存はカラー指定なのでBGRに戻す必要あり）
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    out.write(frame)  # フレームを書き込み

cap.release()
out.release()
# cv2.destroyAllWindows()
